BBKNN integration following https://github.com/Teichlab/bbknn

In [ ]:
import scanpy as sc
import bbknn
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
sc.settings.verbosity = 3  
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80)

In [ ]:
import os
os.chdir("path/to/wd")
cwd = os.getcwd()
print(cwd)

Loading merged data sets which were saved as h5ad files in R using sceasy (https://github.com/cellgeni/sceasy):

convertFormat(seurat_obj, from="seurat", to="anndata", main_layer="counts", drop_single_values=FALSE,  outFile='seurat.h5ad')

In [ ]:
adata = sc.read("path/to/seurat.h5ad")

plot unintegrated data

In [ ]:
sc.pl.umap(adata,  color=['orig.ident','method'])

Run BBKNN for batch "orig.ident". As described, cohorts with multiple modi (e.g., methods or depots), were split.

In [ ]:
bbknn.bbknn(adata, batch_key='orig.ident')
sc.tl.umap(adata)
sc.pl.umap(adata, color=['orig.ident','tissue'])

Run leiden clustering. Tested different resolutions based on final integration and decided on resolution=0.3

In [ ]:
sc.tl.leiden(adata, resolution=0.3)
sc.pl.umap(adata, color = 'leiden')

Repeat BBKNN integration using leiden as confounder key

In [ ]:
bbknn.ridge_regression(adata, batch_key=['orig.ident'], confounder_key=['leiden'])
sc.pp.pca(adata)
bbknn.bbknn(adata, batch_key='orig.ident')
sc.tl.umap(adata)
sc.pl.umap(adata, color=['orig.ident','tissue','method'])

Write integrated data to h5ad, to be translated back into seurat using sceasy

In [ ]:
adata.write("BBKNN_adata.h5ad")